# 0. Imports

In [1]:
import numpy as np
import awkward as ak
import pandas as pd
import uproot as up
import vector
vector.register_awkward()
from matplotlib import pyplot as plt
K = 40000000 * (2750 / 3564)
SCALE = 12
import mplhep as hep
plt.style.use(hep.style.CMS)

# 1. Utility functions

In [2]:
def getBranch(data, l1jet):
    
    # Get list of branches relating to the given jet
    print("Getting list of interesting branches")
    interestingBranches = [branch for branch in data.keys() if l1jet in branch and not f"n{l1jet}" in branch]# and not f"{l1jet}_dau0" in branch] 
    
    # Get only branches corresponding to the desired l1jet
    print("Querying the array with interesting branches")
    l1jetData = data.arrays(interestingBranches)
    
    # Get a dictionary relating default branch names to new branch names (ie without leading ak8PuppiJets_...)
    print("Splitting branch names on _ to get renamed fields")
    renamedFields = {field : field.split('_', maxsplit=1)[-1] for field in interestingBranches}

    # Create a new awkward array with the desired l1jet branches and the new branch names
    print("Returning an ak array of the relevant data with the renamed fields")
    arr = ak.Array({renamedFields[field]: l1jetData[field] for field in l1jetData.fields})
    
    array_dict = {key: arr[key] for key in arr.fields}
    quarks4mom = ak.zip(array_dict)
    quarks4mom = ak.with_name(quarks4mom, "Momentum4D")
    
    return quarks4mom

def get_rate(mask, mb_decisions):
    # leading = ak.firsts(minbias_jets)
    rate = K * ak.sum( mask ) / len(mb_decisions)
    #(leading.mass > mass_threshold) & (leading.pt > pt_threshold)
    
    if isinstance(mb_decisions, pd.core.frame.DataFrame):
        already_passed = mb_decisions.any(axis=1).values
        not_passed_mask = mask[~already_passed]
        return rate, K * ak.sum( not_passed_mask ) / len(mb_decisions)
    else:
        return rate
    
def get_sc4_separated(sc8, sc4, N=1):
    a, b = ak.unzip( ak.cartesian([sc4, sc8[:,:N]], nested=True) )
    dists_bool = a.deltaR(b) < 0.8
    return sc4[~ak.any(dists_bool, axis=-1)]

def get_efficiency(mask, sig_decisions):
    already_passed = sig_decisions.any(axis=1).values
    total_eff = ak.sum( already_passed | mask ) / len(sig_decisions)
    pure_eff = total_eff - ( ak.sum( already_passed ) / len(sig_decisions) )
    return total_eff, pure_eff

# 2. Load data

In [3]:
path_mb_decisions = "/eos/user/l/lroberts/Phase2-L1MenuTools/outputs/V49nano_AR25/rate_tables/MinBias_RateTable_V49nano_AR25_masks.parquet"
mb_decisions = pd.read_parquet(path_mb_decisions, engine="fastparquet").astype(bool)
exclude = ["L1_SinglePfJet8_230"]
triggers_to_load = list(mb_decisions.columns.difference(exclude))
mb_decisions = mb_decisions[triggers_to_load]

path_mb = "/eos/cms/store/group/dpg_trigger/comm_trigger/L1Trigger/roward/phase2/menu/ntuples/Spring24/151X_AllAR25_FindOn/v45/MinBias_TuneCP5_14TeV-pythia8/MinBias_Spring24_200PUALCA_V45_reL1wTT_151X_AllAR25_FindOn/250615_160052/hadd/output_Phase2_L1T.root"
mb = up.open(path_mb + ":Events")
mb_sc8 = getBranch(mb, "L1puppiJetSC8_")
mb_sc4 = getBranch(mb, "L1puppiJetSC4_")

mb_sc4_separated = get_sc4_separated(mb_sc8, mb_sc4, N=1)

Getting list of interesting branches
Querying the array with interesting branches
Splitting branch names on _ to get renamed fields
Returning an ak array of the relevant data with the renamed fields
Getting list of interesting branches
Querying the array with interesting branches
Splitting branch names on _ to get renamed fields
Returning an ak array of the relevant data with the renamed fields


In [56]:
path_sig_decisions = "/eos/user/l/lroberts/Phase2-L1MenuTools/outputs/V49nano_AR25/rate_tables/LH_40_qq_RateTable_V49nano_AR25_masks.parquet"
sig_decisions = pd.read_parquet(path_sig_decisions, engine="fastparquet").astype(bool)
sig_decisions = sig_decisions[triggers_to_load]

path_sig = "/eos/user/l/lroberts/Phase2-L1MenuTools/signal_samples/XtoHH_MH-40_qq_PU200_signal.root"
sig = up.open(path_sig + ":Events")
sig_sc8 = getBranch(sig, "L1puppiJetSC8_")
sig_sc4 = getBranch(sig, "L1puppiJetSC4_")

sig_sc4_separated = get_sc4_separated(sig_sc8, sig_sc4, N=1)

Getting list of interesting branches
Querying the array with interesting branches
Splitting branch names on _ to get renamed fields
Returning an ak array of the relevant data with the renamed fields
Getting list of interesting branches
Querying the array with interesting branches
Splitting branch names on _ to get renamed fields
Returning an ak array of the relevant data with the renamed fields


# 3. Menu seeds

In [5]:
def L1_SinglePFJet8(sc8, sc4, pt_threshold):
    leading = ak.firsts(sc8)
    mask = ak.ones_like(leading.pt, dtype=bool)

    mask = mask & ( leading.pt > pt_threshold )

    return mask


def L1_SinglePFJet8_Mass(sc8, sc4, pt_threshold, mass_threshold):
    leading = ak.firsts(sc8)
    mask = ak.ones_like(leading.pt, dtype=bool)

    mask = mask & ( leading.pt > pt_threshold )
    mask = mask & ( leading.mass > mass_threshold )

    return mask

In [6]:
def L1_DoublePFJet8_dEtaMax(sc8, sc4, pt_thresholdA, pt_thresholdB, separation = 1.6):
    leading = ak.firsts(sc8)
    subleading = ak.firsts(sc8[:,1:])
    mask = ak.ones_like(leading.pt, dtype=bool)
    
    mask = mask & ( abs( leading.eta - subleading.eta ) < separation )    # eta separation
    mask = mask & ( leading.pt > pt_thresholdA )
    mask = mask & ( subleading.pt > pt_thresholdB )
    
    return mask


def L1_DoublePFJet8_dEtaMax_Mass(sc8, sc4, pt_thresholdA, mass_thresholdA, pt_thresholdB, mass_thresholdB, separation = 1.6):
    leading = ak.firsts(sc8)
    subleading = ak.firsts(sc8[:,1:])
    mask = ak.ones_like(leading.pt, dtype=bool)
    
    mask = mask & ( abs( leading.eta - subleading.eta ) < separation )    # eta separation
    mask = mask & ( leading.pt > pt_thresholdA )
    mask = mask & ( leading.mass > mass_thresholdA )
    mask = mask & ( subleading.pt > pt_thresholdB )
    mask = mask & ( subleading.mass > mass_thresholdB )
    
    return mask

In [7]:
def L1_DoublePFJet8_Sum(sc8, sc4, pt_sum_threshold):
    leading = ak.firsts(sc8)
    subleading = ak.firsts(sc8[:,1:])
    mask = ak.ones_like(leading.pt, dtype=bool)
    
    pt_sum = leading.pt + subleading.pt
    
    mask = mask & ( pt_sum > pt_sum_threshold )
    
    return mask


def L1_DoublePFJet8_Sum_Mass(sc8, sc4, pt_sum_threshold, mass_sum_threshold):
    leading = ak.firsts(sc8)
    subleading = ak.firsts(sc8[:,1:])
    mask = ak.ones_like(leading.pt, dtype=bool)
    
    pt_sum = leading.pt + subleading.pt
    mass_sum = leading.mass + subleading.mass
    
    mask = mask & ( pt_sum > pt_sum_threshold )
    mask = mask & ( mass_sum > mass_sum_threshold )
    
    return mask

In [52]:
def L1_DoublePFJet8_Emyr(sc8, sc4, pt_thresholdA, pt_thresholdB):
    leading = ak.firsts(sc8)
    subleading = ak.firsts(sc8[:,1:])
    mask = ak.ones_like(leading.pt, dtype=bool)
    
    mask = mask & (leading.pt > pt_thresholdA)
    mask = mask & (subleading.pt > pt_thresholdB)
    
    return mask


def L1_DoublePFJet8_Emyr_Mass(sc8, sc4, mass_sum_threshold, pt_thresholdA, pt_thresholdB):
    leading = ak.firsts(sc8)
    subleading = ak.firsts(sc8[:,1:])
    mask = ak.ones_like(leading.pt, dtype=bool)
    mass_sum = leading.mass + subleading.mass

    mask = mask & (mass_sum > mass_sum_threshold)
    mask = mask & (leading.pt > pt_thresholdA)
    mask = mask & (subleading.pt > pt_thresholdB)
    
    return mask


def L1_DoublePFJet8_Sum_Mass_dEtaMax(sc8, sc4, mass_sum_threshold, pt_thresholdA, pt_thresholdB, separation=1.6):
    leading = ak.firsts(sc8)
    subleading = ak.firsts(sc8[:,1:])
    mask = ak.ones_like(leading.pt, dtype=bool)
    mass_sum = leading.mass + subleading.mass

    mask = mask & (mass_sum > mass_sum_threshold)
    mask = mask & (leading.pt > pt_thresholdA)
    mask = mask & (subleading.pt > pt_thresholdB)
    mask = mask & ( abs(leading.eta - subleading.eta) < separation )
    
    return mask


In [9]:
def L1_DoublePFJet8_Sum_EtaMax(sc8, sc4, pt_sum_threshold, separation = 1.6):
    leading = ak.firsts(sc8)
    subleading = ak.firsts(sc8[:,1:])
    mask = ak.ones_like(leading.pt, dtype=bool)
    
    mask = mask & ( (leading.pt + subleading.pt) > pt_sum_threshold )
    mask = mask & ( abs( leading.eta - subleading.eta ) < separation )    # eta separation
    
    return mask


def L1_DoublePFJet8_Sum_EtaMax_Mass(sc8, sc4, pt_sum_threshold, mass_sum_threshold, separation = 1.6):
    leading = ak.firsts(sc8)
    subleading = ak.firsts(sc8[:,1:])
    mask = ak.ones_like(leading.pt, dtype=bool)
    
    pt_sum = leading.pt + subleading.pt
    mass_sum = leading.mass + subleading.mass
    
    mask = mask & ( pt_sum > pt_sum_threshold )
    mask = mask & ( mass_sum > mass_sum_threshold )
    mask = mask & ( abs( leading.eta - subleading.eta ) < separation )    # eta separation
    
    return mask

In [10]:
def L1_SinglePFJet8_SinglePFJet4(sc8, sc4, pt_thresholdA, pt_thresholdB):
    leading_sc8 = ak.firsts(sc8)
    leading_sc4 = ak.firsts(sc4)
    mask = ak.ones_like(leading_sc8.pt, dtype=bool)

    mask = mask & ( leading_sc8.pt > pt_thresholdA )
    mask = mask & ( leading_sc4.pt > pt_thresholdB )

    return mask


def L1_SinglePFJet8_SinglePFJet4_Mass(sc8, sc4, pt_thresholdA, mass_thresholdA, pt_thresholdB):
    leading_sc8 = ak.firsts(sc8)
    leading_sc4 = ak.firsts(sc4)
    mask = ak.ones_like(leading_sc8.pt, dtype=bool)

    mask = mask & ( leading_sc8.pt > pt_thresholdA )
    mask = mask & ( leading_sc8.mass > mass_thresholdA )
    mask = mask & ( leading_sc4.pt > pt_thresholdB )

    return mask

In [11]:
def L1_SinglePFJet8_DoublePFJet4(sc8, sc4, pt_thresholdA, pt_thresholdB, pt_thresholdC):
    leading_sc8 = ak.firsts(sc8)
    leading_sc4 = ak.firsts(sc4)
    subleading_sc4 = ak.firsts(sc4[:,1:])
    mask = ak.ones_like(leading_sc8.pt, dtype=bool)

    mask = mask & ( leading_sc8.pt > pt_thresholdA )
    mask = mask & ( leading_sc4.pt > pt_thresholdB )
    mask = mask & ( subleading_sc4.pt > pt_thresholdC )

    return mask


def L1_SinglePFJet8_DoublePFJet4_Mass(sc8, sc4, pt_thresholdA, mass_thresholdA, pt_thresholdB, pt_thresholdC):
    leading_sc8 = ak.firsts(sc8)
    leading_sc4 = ak.firsts(sc4)
    subleading_sc4 = ak.firsts(sc4[:,1:])
    mask = ak.ones_like(leading_sc8.pt, dtype=bool)

    mask = mask & ( leading_sc8.pt > pt_thresholdA )
    mask = mask & ( leading_sc8.mass > mass_thresholdA )
    mask = mask & ( leading_sc4.pt > pt_thresholdB )
    mask = mask & ( subleading_sc4.pt > pt_thresholdC )
    
    return mask

# 4. Wide cone jet seeds analysis

In [12]:
def rate_eff(trigger_func, *thresholds):
    total_rate, pure_rate = get_rate( trigger_func(mb_sc8, mb_sc4_separated, *thresholds), mb_decisions )
    total_eff, pure_eff = get_efficiency( trigger_func(sig_sc8, sig_sc4_separated, *thresholds), sig_decisions )
    print(f"{trigger_func.__name__}{thresholds}:\n    Total rate = {total_rate/1000:.2f} kHz, pure rate: {pure_rate/1000:.2f} kHz\n    Total eff: {total_eff*100:.2f}%, pure eff: {pure_eff*100:.2f}%")
    return total_rate, pure_rate, total_eff, pure_eff

In [ ]:
trigger = L1_DoublePFJet8_Sum_Mass_dEtaMax
mass_sum_threshold = 32.87
pt_thresholdA = 84.98
pt_thresholdB = 73.77
_ = rate_eff(trigger, mass_sum_threshold, pt_thresholdA, pt_thresholdB)

In [ ]:
trigger = L1_DoublePFJet8_Emyr
pt_thresholdA = 123.3    
pt_thresholdB = 50.83    # optimised 20 gev H    
_ = rate_eff(trigger, pt_thresholdA, pt_thresholdB)

trigger = L1_DoublePFJet8_Emyr_Mass
mass_sum_threshold = 32.87
pt_thresholdA = 84.98
pt_thresholdB = 73.77
_ = rate_eff(trigger, mass_sum_threshold, pt_thresholdA, pt_thresholdB)

L1_DoublePFJet8_Emyr(123.3, 50.83):
    Total rate = 80.69 kHz, pure rate: 9.82 kHz
    Total eff: 77.30%, pure eff: 2.51%
L1_DoublePFJet8_Emyr_Mass(32.87, 84.98, 73.77):
    Total rate = 53.09 kHz, pure rate: 9.96 kHz
    Total eff: 80.40%, pure eff: 5.61%


In [14]:
trigger = L1_SinglePFJet8
pt_threshold = 126.5    # optimised
_ = rate_eff(trigger, pt_threshold)

trigger = L1_SinglePFJet8_Mass
pt_threshold = 120.7    # optimised
mass_threshold = 17.37    # optimised
_ = rate_eff(trigger, pt_threshold, mass_threshold)

L1_SinglePFJet8(126.5,):
    Total rate = 86.03 kHz, pure rate: 9.82 kHz
    Total eff: 76.92%, pure eff: 2.13%
L1_SinglePFJet8_Mass(120.7, 17.37):
    Total rate = 54.58 kHz, pure rate: 9.71 kHz
    Total eff: 77.89%, pure eff: 3.09%


In [15]:
# MOST EFFICIENT

trigger = L1_DoublePFJet8_dEtaMax
pt_thresholdA = 82.78    # optimised
pt_thresholdB = 74.44    # optimised
_ = rate_eff(trigger, pt_thresholdA, pt_thresholdB)

trigger = L1_DoublePFJet8_dEtaMax_Mass
pt_thresholdA = 71.96    # optimised
mass_thresholdA = 15.60    # optimised
pt_thresholdB = 60.04    # optimised
mass_thresholdB = 12.91    # optimised
_ = rate_eff(trigger, pt_thresholdA, mass_thresholdA, pt_thresholdB, mass_thresholdB)

L1_DoublePFJet8_dEtaMax(82.78, 74.44):
    Total rate = 77.44 kHz, pure rate: 9.99 kHz
    Total eff: 81.24%, pure eff: 6.45%
L1_DoublePFJet8_dEtaMax_Mass(71.96, 15.6, 60.04, 12.91):
    Total rate = 33.93 kHz, pure rate: 9.88 kHz
    Total eff: 82.27%, pure eff: 7.48%


In [16]:
# MOST EFFICIENT MH50, MH40

trigger = L1_DoublePFJet8_Sum
pt_sum_threshold = 201.0    # optimised
_ = rate_eff(trigger, pt_sum_threshold)

trigger = L1_DoublePFJet8_Sum_Mass
pt_sum_threshold = 177.8    # optimised
mass_sum_threshold = 34.01    # optimised
_ = rate_eff(trigger, pt_sum_threshold, mass_sum_threshold)

L1_DoublePFJet8_Sum(201.0,):
    Total rate = 92.54 kHz, pure rate: 9.94 kHz
    Total eff: 78.01%, pure eff: 3.22%
L1_DoublePFJet8_Sum_Mass(177.8, 34.01):
    Total rate = 53.42 kHz, pure rate: 9.99 kHz
    Total eff: 80.34%, pure eff: 5.54%


In [17]:
# MOST EFFICIENT AND MOST EFFICIENT FOR MH50

trigger = L1_DoublePFJet8_Sum_EtaMax
pt_sum_threshold = 180.0    # optimised
separation = 1.6
_ = rate_eff(trigger, pt_sum_threshold, separation)

trigger = L1_DoublePFJet8_Sum_EtaMax_Mass
pt_sum_threshold = 166.1
mass_sum_threshold = 30.92
separation = 1.6
_ = rate_eff(trigger, pt_sum_threshold, mass_sum_threshold, separation)

L1_DoublePFJet8_Sum_EtaMax(180.0, 1.6):
    Total rate = 75.31 kHz, pure rate: 9.99 kHz
    Total eff: 80.98%, pure eff: 6.19%
L1_DoublePFJet8_Sum_EtaMax_Mass(166.1, 30.92, 1.6):
    Total rate = 46.62 kHz, pure rate: 9.89 kHz
    Total eff: 82.85%, pure eff: 8.06%


In [18]:
# trigger = L1_SinglePFJet8_SinglePFJet4
# pt_thresholdA = 120.8    # optimised
# pt_thresholdB = 53.02    # optimised
# _ = rate_eff(trigger, pt_thresholdA, pt_thresholdB)

# trigger = L1_SinglePFJet8_SinglePFJet4_Mass
# pt_thresholdA = 82.23    # optimised
# mass_thresholdA = 24.55    # optimised
# pt_thresholdB = 52.74    # optimised
# _ = rate_eff(trigger, pt_thresholdA, mass_thresholdA, pt_thresholdB)

In [19]:
# trigger = L1_SinglePFJet8_DoublePFJet4
# pt_thresholdA = 121.0    # optimised
# pt_thresholdB = 52.87    # optimised
# pt_thresholdC = 4.758    # optimised
# _ = rate_eff(trigger, pt_thresholdA, pt_thresholdB, pt_thresholdC)

# trigger = L1_SinglePFJet8_DoublePFJet4_Mass
# pt_thresholdA = 84.36    # optimised
# mass_thresholdA = 24.59    # optimised
# pt_thresholdB = 51.52    # optimised
# pt_thresholdC = 9.019    # optimised
# _ = rate_eff(trigger, pt_thresholdA, mass_thresholdA, pt_thresholdB, pt_thresholdC)

# 5. Optimisation

In [50]:
from scipy.optimize import differential_evolution as de
def objective(x, trigger):
    total_rate, pure_rate, total_eff, pure_eff = rate_eff(trigger, *x)
    if pure_rate > 10000:
        return pure_eff
    else:
        return -pure_eff

In [57]:
#L1_DoublePFJet8_Sum_Mass_dEtaMax
bounds = ((0, 70), (30, 160), (20, 150))
L1_DoublePFJet8_Sum_Mass_dEtaMax_Opti = de(objective, bounds=bounds, args=(L1_DoublePFJet8_Sum_Mass_dEtaMax,), workers=-1, disp=True)
L1_DoublePFJet8_Sum_Mass_dEtaMax_Opti

/eos/home-l/lroberts/Phase2-L1MenuTools/data/.env/lib64/python3.9/site-packages/scipy/optimize/_differentialevolution.py:487: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


L1_DoublePFJet8_Sum_Mass_dEtaMax(np.float64(42.791638836310284), np.float64(140.3134523247012), np.float64(31.731481907481147)):
    Total rate = 10.10 kHz, pure rate: 0.31 kHz
    Total eff: 92.67%, pure eff: 0.86%
L1_DoublePFJet8_Sum_Mass_dEtaMax(np.float64(20.025240206370754), np.float64(85.24489309583312), np.float64(130.24009159005905)):
    Total rate = 9.69 kHz, pure rate: 0.03 kHz
    Total eff: 91.93%, pure eff: 0.12%
L1_DoublePFJet8_Sum_Mass_dEtaMax(np.float64(29.960286947330633), np.float64(120.24207225891777), np.float64(140.606922047515)):
    Total rate = 6.33 kHz, pure rate: 0.00 kHz
    Total eff: 91.93%, pure eff: 0.12%
L1_DoublePFJet8_Sum_Mass_dEtaMax(np.float64(31.359872356929003), np.float64(128.2990532697526), np.float64(81.53178311736438)):
    Total rate = 18.87 kHz, pure rate: 0.42 kHz
    Total eff: 92.98%, pure eff: 1.17%
L1_DoublePFJet8_Sum_Mass_dEtaMax(np.float64(15.3344800448587), np.float64(59.04255036644681), np.float64(77.27269742312457)):
    Total rate

             message: Optimization terminated successfully.
             success: True
                 fun: -0.04064039408866993
                   x: [ 3.617e+01  7.712e+01  5.166e+01]
                 nit: 34
                nfev: 1579
          population: [[ 3.617e+01  7.712e+01  5.166e+01]
                       [ 3.595e+01  7.513e+01  5.605e+01]
                       ...
                       [ 3.609e+01  7.895e+01  5.042e+01]
                       [ 3.611e+01  7.988e+01  5.114e+01]]
 population_energies: [-4.064e-02 -4.064e-02 ... -4.002e-02 -4.002e-02]

In [ ]:
# # L1_SinglePFJet8_Mass
# bounds = ((0, 70), (30, 160), (20, 150))
# L1_SinglePFJet8_Mass_Opti = de(objective, bounds=bounds, args=(L1_DoublePFJet8_Emyr_Mass,), workers=-1, disp=True)
# L1_SinglePFJet8_Mass_Opti

/eos/home-l/lroberts/Phase2-L1MenuTools/data/.env/lib64/python3.9/site-packages/scipy/optimize/_differentialevolution.py:487: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


L1_DoublePFJet8_Emyr_Mass(np.float64(31.765168145532265), np.float64(85.00816952131274), np.float64(86.53171197982033)):
    Total rate = 42.33 kHz, pure rate: 4.49 kHz
    Total eff: 97.32%, pure eff: 1.76%
L1_DoublePFJet8_Emyr_Mass(np.float64(47.99552950017241), np.float64(158.54011021329256), np.float64(69.50899837216319)):
    Total rate = 9.24 kHz, pure rate: 0.08 kHz
    Total eff: 95.76%, pure eff: 0.19%
L1_DoublePFJet8_Emyr_Mass(np.float64(61.586356522247456), np.float64(65.68111219117782), np.float64(133.5123199926599)):
    Total rate = 3.90 kHz, pure rate: 0.03 kHz
    Total eff: 95.64%, pure eff: 0.08%
L1_DoublePFJet8_Emyr_Mass(np.float64(25.89564631180885), np.float64(74.5341979891144), np.float64(30.955056678016653)):
    Total rate = 226.08 kHz, pure rate: 126.66 kHz
    Total eff: 99.04%, pure eff: 3.48%
L1_DoublePFJet8_Emyr_Mass(np.float64(17.71719436244662), np.float64(100.47759436320868), np.float64(61.920865381648795)):
    Total rate = 117.91 kHz, pure rate: 27.99 

             message: Optimization terminated successfully.
             success: True
                 fun: -0.026376146788990806
                   x: [ 3.675e+01  9.383e+01  5.707e+01]
                 nit: 32
                nfev: 1489
          population: [[ 3.675e+01  9.383e+01  5.707e+01]
                       [ 3.751e+01  9.231e+01  5.557e+01]
                       ...
                       [ 3.677e+01  9.429e+01  5.683e+01]
                       [ 3.651e+01  9.490e+01  5.923e+01]]
 population_energies: [-2.638e-02 -2.638e-02 ... -2.638e-02 -2.599e-02]

In [23]:
# # L1_SinglePFJet8_Mass
# bounds = ((80, 130), (10, 40))
# L1_SinglePFJet8_Mass_Opti = de(objective, bounds=bounds, args=(L1_SinglePFJet8_Mass,), workers=-1, disp=True)
# L1_SinglePFJet8_Mass_Opti

In [24]:
# # L1_DoublePFJet8_dEtaMax
# bounds = ((50, 130), (40, 120))
# L1_DoublePFJet8_dEtaMax_Opti = de(objective, bounds=bounds, args=(L1_DoublePFJet8_dEtaMax,), workers=-1, disp=True)
# L1_DoublePFJet8_dEtaMax_Opti

In [25]:
# # L1_DoublePFJet8_dEtaMax_Mass
# bounds = ((50, 130), (0, 30), (40, 120), (0, 30) )
# L1_DoublePFJet8_dEtaMax_Mass_Opti = de(objective, bounds=bounds, args=(L1_DoublePFJet8_dEtaMax_Mass,), workers=-1, disp=True)
# L1_DoublePFJet8_dEtaMax_Mass_Opti

In [26]:
# # L1_DoublePFJet8_Sum_Mass
# bounds = ((0, 205), (0, 60) )
# L1_DoublePFJet8_Sum_Mass_Opti = de(objective, bounds=bounds, args=(L1_DoublePFJet8_Sum_Mass,), workers=-1, disp=True)
# L1_DoublePFJet8_Sum_Mass_Opti

In [27]:
# # L1_DoublePFJet8_Sum_EtaMax_Mass
# bounds = ((0, 205), (0, 60) )
# L1_DoublePFJet8_Sum_EtaMax_Mass_Opti = de(objective, bounds=bounds, args=(L1_DoublePFJet8_Sum_EtaMax_Mass,), workers=-1, disp=True)
# L1_DoublePFJet8_Sum_EtaMax_Mass_Opti

In [28]:
# # L1_SinglePFJet8_SinglePFJet4
# bounds = ((0, 130), (0, 130) )
# L1_SinglePFJet8_SinglePFJet4_Opti = de(objective, bounds=bounds, args=(L1_SinglePFJet8_SinglePFJet4,), workers=-1, disp=True)
# L1_SinglePFJet8_SinglePFJet4_Opti

In [29]:
# # L1_SinglePFJet8_SinglePFJet4_Mass
# bounds = ((0, 130), (0, 30), (0, 130) )
# L1_SinglePFJet8_SinglePFJet4_Mass_Opti = de(objective, bounds=bounds, args=(L1_SinglePFJet8_SinglePFJet4_Mass,), workers=-1, disp=True)
# L1_SinglePFJet8_SinglePFJet4_Mass_Opti

In [30]:
# # L1_SinglePFJet8_DoublePFJet4
# bounds = ((0, 130), (0, 130), (0, 130) )
# L1_SinglePFJet8_DoublePFJet4_Opti = de(objective, bounds=bounds, args=(L1_SinglePFJet8_DoublePFJet4,), workers=-1, disp=True)
# L1_SinglePFJet8_DoublePFJet4_Opti

In [31]:
# # L1_SinglePFJet8_DoublePFJet4_Mass
# bounds = ((0, 130), (0, 30), (0, 130), (0, 130) )
# L1_SinglePFJet8_DoublePFJet4_Mass_Opti = de(objective, bounds=bounds, args=(L1_SinglePFJet8_DoublePFJet4_Mass,), workers=-1, disp=True)
# L1_SinglePFJet8_DoublePFJet4_Mass_Opti

# 6. Investigate trigger performance

### 6.1 Turnon curves

In [32]:
def get_turnon(mask, sc8):
    bins = np.linspace(0, 300, 31)
    xs = 0.5 * (bins[1:] + bins[:-1])
    effs, errs, counts = [], [], []
    for b in range(len(bins)-1):
        inBinMask = (sc8[f"genmass"] >= bins[b]) & (sc8[f"genmass"] < bins[b+1])
        denominator = ak.sum( inBinMask )
        numerator = ak.sum( mask & inBinMask )

        eff = numerator / denominator if denominator > 0 else 0
        err = np.sqrt(eff * (1-eff) / denominator) if denominator > 0 else 0

        effs.append(eff)
        errs.append(err)
        counts.append(denominator)

    return xs, np.array(effs), np.array(errs), np.array(counts)

In [33]:
def plot_turnon(xs, formatting, counts, **effs):
    plt.figure(figsize=(SCALE,2*SCALE/3))
    plt.bar(x=xs, height = counts/max(counts), width=xs[1]-xs[0], color="lightgrey", alpha=0.5, align="center")
    for label, (eff, err) in effs.items():
        plt.errorbar(xs, eff, xerr=xs[0], yerr=err,
                     capsize=5, fmt="-o", label=formatting[label])

    plt.legend()
    plt.grid()
    plt.show()

In [34]:
# formatting = {
#     "Full_menu": "Full menu",
#     "Full_and_L1_DoublePFJet8_dEtaMax_Mass": "L1_DoublePFJet8_dEtaMax_Mass"
#     }

# # full_menu_mask = sig_decisions.any(axis=1).values
# # xs, effs_baseline, errs_baseline, counts = get_turnon(full_menu_mask, sig_sc8)

# mask = L1_DoublePFJet8_dEtaMax_Mass(sig_sc8, None, 71.95889629199519, 15.595871084158233, 60.04119874222349, 12.905588652812998)
# xs, effs, errs, counts = get_turnon( mask, sig_sc8)

# plot_turnon(
#     xs, formatting=formatting, counts=counts,
#     # Full_menu=(effs_baseline, errs_baseline),
#     Full_and_L1_DoublePFJet8_dEtaMax_Mass=(effs, errs)
#     )